### Import Libraries

In [ ]:
import pandas as pd
from transformers import pipeline
from google.colab import files
from io import StringIO

### Upload Data

In [ ]:
uploaded = files.upload()

Saving study.csv to study.csv


In [ ]:
data = uploaded

In [ ]:
# Get the CSV byte string from the dictionary (assuming there's only one key or you know the key name)
csv_bytes = data['study.csv']

# Decode the bytes to a string
csv_str = csv_bytes.decode('utf-8')

# Use StringIO to turn the string into a buffer
csv_buffer = StringIO(csv_str)

# Create a DataFrame
df = pd.read_csv(csv_buffer)

In [ ]:
df

,Timestamp,Score,"What tools or resources do you find most helpful when you study (e.g., flashcards, apps, websites)?","Do you take breaks while studying? If so, what do you usually do during breaks?",How do you keep track of assignment due dates and exam dates?,How do you prepare for upcoming tests or exams?,"What challenges do you face when studying, and how do you overcome them?",Unnamed: 7,Unnamed: 8
0,4/14/2024 16:14:13,NaN,No,Sleep,Germany,LearnPlay,NaN,"No,Sleep,Germany,LearnPlay,",Good Student
1,4/14/2024 16:40:52,NaN,I like to do watercolor paintings especially ...,I like to play on my switch,id want to go to iceland so I can enjoy natur...,no,NaN,I like to do watercolor paintings especially ...,Good Student
2,4/14/2024 16:42:11,NaN,No none at all,Watch tv shows,I would go to Japan and I would like to explor...,I am a part of the Columbia University Neurosc...,NaN,"No none at all,Watch tv shows ,I would go to J...",Good Student
3,4/14/2024 16:44:52,NaN,no I hate art,I like to drink alcohol and sleep,jail,the I hate homework club,NaN,"no I hate art ,I like to drink alcohol and s...",Bad Student
4,4/14/2024 16:47:11,NaN,I enjoy writing poetry.,My favorite way to relax or destress is to bak...,I would love to travel to Japan or Thailand.,I am a part of LearnPlay which I love because ...,NaN,"I enjoy writing poetry. ,My favorite way to re...",Good Student
...,...,...,...,...,...,...,...,...,...
122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


### Loading Masking Model

In [ ]:
unmasker = pipeline("fill-mask", model="bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
result = unmasker("This man works as a [MASK].")
print([r["token_str"] for r in result])

['carpenter', 'lawyer', 'farmer', 'businessman', 'doctor']


In [ ]:
result = unmasker("This woman works as a [MASK].")
print([r["token_str"] for r in result])

['nurse', 'maid', 'teacher', 'waitress', 'prostitute']


### Rename Columns for Easy Use

In [ ]:
df.rename(columns = {"What tools or resources do you find most helpful when you study (e.g., flashcards, apps, websites)?":'tools',
                     "Do you take breaks while studying? If so, what do you usually do during breaks?":"breaks",
                     "How do you keep track of assignment due dates and exam dates?":"tracking",
                     "How do you prepare for upcoming tests or exams?": "prep",
                     "What challenges do you face when studying, and how do you overcome them?":"challenge"}, inplace = True)

In [ ]:
df = df[['tools', 'breaks','tracking','prep', 'challenge']]

### Generate New Column for Input - 'Sentence'

In [ ]:
# Using apply function to create a new column
df['Sentence'] = df.apply(
    lambda row: (
        "A student said the following about studying tools: " + str(row["tools"]) + ". " +
        "A student said the following about studying breaks: " + str(row["breaks"]) + ". " +
        "A student said the following about due dates and exams: " + str(row["tracking"]) + ". " +
        "A student said the following about test preparation: " + str(row["prep"]) + ". " +
        "A student said the following about studying challenges: " + str(row["challenge"]) + ". " +
        "Based on this info, this student should work as a [MASK]."
    ), axis=1
)

<ipython-input-12-e97f0118f214>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sentence'] = df.apply(


In [ ]:
df.head()

,tools,breaks,tracking,prep,challenge,Sentence
0,No,Sleep,Germany,LearnPlay,NaN,A student said the following about studying to...
1,I like to do watercolor paintings especially ...,I like to play on my switch,id want to go to iceland so I can enjoy natur...,no,NaN,A student said the following about studying to...
2,No none at all,Watch tv shows,I would go to Japan and I would like to explor...,I am a part of the Columbia University Neurosc...,NaN,A student said the following about studying to...
3,no I hate art,I like to drink alcohol and sleep,jail,the I hate homework club,NaN,A student said the following about studying to...
4,I enjoy writing poetry.,My favorite way to relax or destress is to bak...,I would love to travel to Japan or Thailand.,I am a part of LearnPlay which I love because ...,NaN,A student said the following about studying to...


### Masking Function and Application - 'prediction_professions'

In [ ]:
def apply_unmasking(masked_text):
    # Perform the mask filling
    results = unmasker(masked_text)
    # Extract only the token strings, discarding the scores
    token_strs = [result["token_str"] for result in results]
    return token_strs

# Apply the transformation to the 'masked_profession' column and store results in a new column
df['predicted_professions'] = df['Sentence'].apply(apply_unmasking)

<ipython-input-14-be8b85fbc215>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['predicted_professions'] = df['Sentence'].apply(apply_unmasking)


In [ ]:
df

,tools,breaks,tracking,prep,challenge,Sentence,predicted_professions
0,No,Sleep,Germany,LearnPlay,NaN,A student said the following about studying to...,"[teacher, trainer, coach, professional, tutor]"
1,I like to do watercolor paintings especially ...,I like to play on my switch,id want to go to iceland so I can enjoy natur...,no,NaN,A student said the following about studying to...,"[teacher, designer, trainer, professional, coach]"
2,No none at all,Watch tv shows,I would go to Japan and I would like to explor...,I am a part of the Columbia University Neurosc...,NaN,A student said the following about studying to...,"[teacher, researcher, mentor, trainer, psychol..."
3,no I hate art,I like to drink alcohol and sleep,jail,the I hate homework club,NaN,A student said the following about studying to...,"[teacher, professional, coach, trainer, journa..."
4,I enjoy writing poetry.,My favorite way to relax or destress is to bak...,I would love to travel to Japan or Thailand.,I am a part of LearnPlay which I love because ...,NaN,A student said the following about studying to...,"[teacher, coach, trainer, tutor, designer]"
...,...,...,...,...,...,...,...
122,NaN,NaN,NaN,NaN,NaN,A student said the following about studying to...,"[teacher, trainer, professional, coach, resear..."
123,NaN,NaN,NaN,NaN,NaN,A student said the following about studying to...,"[teacher, trainer, professional, coach, resear..."
124,NaN,NaN,NaN,NaN,NaN,A student said the following about studying to...,"[teacher, trainer, professional, coach, resear..."
125,NaN,NaN,NaN,NaN,NaN,A student said the following about studying to...,"[teacher, trainer, professional, coach, resear..."
